<a href="https://colab.research.google.com/github/MohammedFarzin/Fine-tuning-llms/blob/main/fine_tuning_llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.32.1 --progress-bar off
!pip install -qqq datasets==2.14.4 --progress-bar off
!pip install -qqq peft==0.5.0 --progress-bar off
!pip install -qqq bitsandbytes==0.41.1 --progress-bar off
!pip install -qqq trl==0.7.1 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.0+cu121 requires torch==2.3.0, but you have torch 2.0.1 which is incompatible.
torchtext 0.18.0 requires torch>=2.3.0, but you have torch 2.0.1 which is incompatible.
torchvision 0.18.0+cu121 requires torch==2.3.0, but you have torch 2.0.1 which is incompatible.


In [ ]:
import json
import re
from pprint import pprint

import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login
from peft import PeftModel, LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
  )
from trl import SFTTrainer

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "meta-llama/Llama-2-7b-hf"

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [ ]:
! huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) 
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in ca

In [ ]:
dataset = load_dataset("Salesforce/dialogstudio", "TweetSumm")
dataset

❤️Attention❤️: Dataset download may take some time. We appreciate your patience!


DatasetDict({
    train: Dataset({
        features: ['original dialog id', 'new dialog id', 'dialog index', 'original dialog info', 'log', 'prompt'],
        num_rows: 879
    })
    validation: Dataset({
        features: ['original dialog id', 'new dialog id', 'dialog index', 'original dialog info', 'log', 'prompt'],
        num_rows: 110
    })
    test: Dataset({
        features: ['original dialog id', 'new dialog id', 'dialog index', 'original dialog info', 'log', 'prompt'],
        num_rows: 110
    })
})

In [ ]:
# summary = json.loads(dataset['train']['original dialog info'][0])["summaries"]['abstractive_summaries'][0]
# summary = " ".join(summary)
# summary

dataset['train']['log'][0][0]



{'turn id': 1,
 'user utterance': 'So neither my iPhone nor my Apple Watch are recording my steps/activity, and Health doesn’t recognise either source anymore for some reason. Any ideas? https://t.co/m9DPQbkftD @AppleSupport please read the above.',
 'system response': '@135060 Let’s investigate this together. To start, can you tell us the software versions your iPhone and Apple Watch are running currently?',
 'dialog history': '',
 'original user side information': '{}',
 'original system side information': '{}'}

In [ ]:
def clean_text(text):
  text = re.sub(r"http\S+", "", text)
  text = re.sub(r"@[^\s]+", "", text)
  text = re.sub(r"\s+", "", text)
  return re.sub(r"\^[^ ]+", "", text)

def create_conversation_text(data_points):
  conversation_text = ""
  for data_point in data_points["log"]:
    user = clean_text(data_point["user"])
    text += f"user: {user.strip()}\n"

    agent = clean_text(data_point["system response"])
    text += f"agent: {agent.strip()}\n"
  return conversation_text

In [ ]:
DEFAULT_SYSTEM_PROMPT = """
Below is a conversation between a human and an AI agent. Write a summary of the conversation.
"""
def generate_training_prompt(conversation_text, summary, system_prompt=DEFAULT_SYSTEM_PROMPT):
  return f"""
  ### Instruction : {sus}"""


In [ ]:
def generate_text(data_point):
  summaries = json.loads(data_point["original dialog info"])["summaries"]['abstractive_summaries']
  summary = summaries[0]
  summary = " ".join(summary)

  conversation_text = create_conversation_text(data_points)
  return {
      "conversation" : conversation_text,
      "summary" : summary,
      "text": generate_training_prompt(conversation_text, summary)
  }
